In [2]:
import pandas as pd
import numpy as np
import tushare as ts
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

from get_data_163 import FinData
import datetime

today = datetime.datetime.today()
cur_year = str(today.year)

# 设置参数

In [3]:
comp_dicts = {
#     "000672":"上峰水泥",
#     "600585":"海螺水泥",
    
#     "300552":"万集科技",
#     "002869":"金溢科技",
    
#     "601318":"中国平安",
    '603589':'口子窖',
#     '000596':'古井贡酒',
#     '603189':'迎驾贡酒',
#     '600199':'金种子酒',
    "002304":'洋河股份'
}


# target_code = '002304' # y洋河

In [4]:
start_year = '2016'
date_filter = "12-31"

In [44]:
data_dfs = {}
for k in comp_dicts:
    fd = FinData(k, date_filter=date_filter)
    
    zycwzb = fd.zycwzb
    lrb = fd.lrb
    zcfzb = fd.zcfzb
    xjllb = fd.xjllb

    data_df = pd.concat([zycwzb, zcfzb, lrb, xjllb], axis=1)

    data_df['grossprofit_margin'] = 100 - data_df['main_cost_ratio'] 

    data_df['cogs_of_sales'] = data_df['main_cost_ratio']

    data_df['saleexp_to_gr'] = data_df['sell_exp']/data_df['revenue']
    data_df['adminexp_of_gr'] = data_df['admin_exp']/data_df['revenue']
    data_df['finaexp_of_gr'] = data_df['fin_exp']/data_df['revenue']

    data_df = data_df.iloc[:,~data_df.columns.duplicated()]

    ## 公允、投资、减值
    data_df['gtj_to_oper'] = data_df[['fv_value_chg_gain','invest_income','assets_impair_loss']].sum(axis=1) / data_df['operate_profit'] *100

    ## 净利/营业利润
    data_df['n_to_oper'] = data_df['n_income'] / data_df['operate_profit'] *100

    data_df['saleexp_to_gr'] = data_df['sell_exp']/data_df['revenue']
    data_df['adminexp_of_gr'] = data_df['admin_exp']/data_df['revenue']
    data_df['finaexp_of_gr'] = data_df['fin_exp']/data_df['revenue']

    data_df['profit_dedt_yoy'] = data_df.profit_dedt/data_df.profit_dedt.shift(-1) - 1
    data_df['revenue_yoy'] = data_df.revenue/data_df.revenue.shift(-1) - 1

    data_df['total_ass_yoy'] = data_df.total_assets/data_df.total_assets.shift(-1)
    data_df['total_hldr_eqy_exc_min_int_yoy'] = data_df.total_hldr_eqy_exc_min_int/data_df.total_hldr_eqy_exc_min_int.shift(-1) - 1

    data_df['salescash_to_or'] = data_df.c_fr_sale_sg / data_df.total_revenue

    data_df['n_income_to_cha'] = data_df['n_cashflow_act']/data_df['n_income'] 
    data_df['profit_dedt_to_cha'] = data_df['n_cashflow_act']/data_df['profit_dedt'] 

    data_dfs[comp_dicts[k]] = data_df

get 利润表 from mysql for 603589 success.
get 资产负债表 from mysql for 603589 success.
get 现金流量表 from mysql for 603589 success.
get 财务报表摘要 from mysql for 603589 success.
get 主要财务指标 from mysql for 603589 success.
get 盈利能力 from mysql for 603589 success.
get 偿还能力 from mysql for 603589 success.
get 成长能力 from mysql for 603589 success.
get 营运能力 from mysql for 603589 success.
get 利润表 from mysql for 002304 success.
get 资产负债表 from mysql for 002304 success.
get 现金流量表 from mysql for 002304 success.
get 财务报表摘要 from mysql for 002304 success.
get 主要财务指标 from mysql for 002304 success.
get 盈利能力 from mysql for 002304 success.
get 偿还能力 from mysql for 002304 success.
get 成长能力 from mysql for 002304 success.
get 营运能力 from mysql for 002304 success.


# ROE 分析

## ROE & ROA

In [45]:
fig = make_subplots(1,2, subplot_titles=['ROE','ROA'])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    fig.append_trace(go.Scatter(x=data_dfs[k].index, y=data_dfs[k].roe,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_dfs[k].index, y=data_dfs[k].roa,name=k,line=line_,legendgroup=k, showlegend=False),1,2)

# for k in data_dfs:
fig.show()

## 杜邦分析

In [46]:
fig = make_subplots(2,2, subplot_titles=['ROE','净利率&毛利率','周转率','资产负债率'])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.roe,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.netprofit_margin,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.assets_turn,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.debt_to_assets,name=k,line=line_,legendgroup=k, showlegend=False),2,2)
    
fig.show()

# 风险分析-结构

## 应收

In [47]:
cols_rec = {"notes_receiv":"应收票据",
"accounts_receiv":"应收账款",
"oth_receiv":"其他应收款",
"lt_amor_exp":"长期待摊费用"}

In [48]:
fig = make_subplots(2,2, subplot_titles=['应收票据','应收账款','其他应收款','长期待摊费用',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.notes_receiv/data_df.total_revenue,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.accounts_receiv/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.oth_receiv/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.lt_amor_exp/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),2,2)
    
fig.show()

## 应付

In [49]:
cols_pay = {"notes_payable":"应付票据",
"acct_payable":"应付账款",
"oth_payable":"其他应付款",
"st_borr":"短期借款",}

In [50]:
fig = make_subplots(2,2, subplot_titles=['应付票据','应付账款','其他应付款','短期借款',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.notes_payable/data_df.total_revenue,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.acct_payable/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.oth_payable/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.st_borr/data_df.total_revenue,name=k,line=line_,legendgroup=k, showlegend=False),2,2)
    
fig.show()

## 核心利润&营业利润

In [ ]:
for i,k in enumerate(data_dfs.keys()):
    data_df = data_dfs[k]
    
    data_df['oper_profit'] = data_df['revenue'] - (
            data_df[["oper_cost", "int_exp", "comm_exp", "biz_tax_surchg", "sell_exp", "admin_exp", "prem_refund",
                     "compens_payout", "reser_insur_liab", "div_payt", "reins_exp", "compens_payout_refu",
                     "insur_reser_refu", "reins_cost_refund", "other_bus_cost"]].sum(axis=1) + 
        data_df['fin_exp'].apply(lambda x: x if x > 0 else 0) + 
        data_df['assets_impair_loss'].apply(lambda x: x if x > 0 else 0)
    )
    ## 暂时可能不适用于金融类公司
    data_df['oper_profit_ratio'] = data_df['oper_profit']/data_df['revenue']*100

    ## 公允、投资、减值
    data_df['gtj_to_oper'] = data_df[['fv_value_chg_gain','invest_income','assets_impair_loss']].sum(axis=1) / data_df['operate_profit'] *100

    ## 净利/营业利润
    data_df['n_to_oper'] = data_df['n_income'] / data_df['operate_profit'] *100

In [ ]:
fig = make_subplots(2,2, subplot_titles=['核心利润率','营业利润率','公投减/营业利润','净利率/营业利润',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.oper_profit_ratio,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.op_of_gr,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.gtj_to_oper,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.n_to_oper,name=k,line=line_,legendgroup=k, showlegend=False),2,2)
    
fig.show()

# 安全性

## 有息负债率

In [51]:
cols_yx_debts = {
    "lt_borr":"长期借款",
"st_borr":"短期借款",
"cb_borr":"向中央银行借款",
"trading_fl":"交易性金融负债",
"sold_for_repur_fa":"卖出回购金融资产款",
"st_bonds_payable":"应付短期债券",
"non_cur_liab_due_1y":"一年内到期的非流动负债",
"oth_cur_liab":"其他流动负债",
"bond_payable":"应付债券",
"oth_ncl":"其他非流动负债",
}
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['debts_yx'] = data_df[cols_yx_debts.keys()].sum(axis=1)

In [52]:
fig = make_subplots(1,2, subplot_titles=['有息负债率','除商誉的有息负债率'])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df['debts_yx']/ data_df['total_assets'],name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df['debts_yx'] / (data_df['total_assets'] - data_df['goodwill']),name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    
fig.show()

## 现金及等价物/有息负债

In [53]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['money_cap']/data_df['debts_yx'],name=k))
fig.show()


# 盈利能力

## 核心利润

In [54]:
fig = make_subplots(2,2, subplot_titles=['销售毛利率','销售净利率','销售成本率','销售期间费用率',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.grossprofit_margin,name=k,line=line_,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.netprofit_margin,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.gtj_to_oper,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.cogs_of_sales,name=k,line=line_,legendgroup=k, showlegend=False),2,2)
    
fig.show()

## 费用占比

In [55]:
fig = make_subplots(2,2, subplot_titles=['营业收入','销售费用率','管理费用率','财务费用率',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Bar(x=data_df.index, y=data_df.revenue,name=k,legendgroup=k),1,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.saleexp_to_gr,name=k,line=line_,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.adminexp_of_gr,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.finaexp_of_gr,name=k,line=line_,legendgroup=k, showlegend=False),2,2)

fig.update_layout(barmode='stack')
fig.show()

# 成长性

## 营收、扣非、净利润增长

In [56]:
fig = make_subplots(2,2, subplot_titles=['营业收入','扣非净利润','营收增长','扣非增长',])

for i,k in enumerate(data_dfs.keys()):
    line_ = dict(color=DEFAULT_PLOTLY_COLORS[i])
    data_df = data_dfs[k]
    
    fig.append_trace(go.Bar(x=data_df.index, y=data_df.total_revenue, marker_color=DEFAULT_PLOTLY_COLORS[i],name=k,legendgroup=k),1,1)
    fig.append_trace(go.Bar(x=data_df.index, y=data_df.n_income, marker_color=DEFAULT_PLOTLY_COLORS[i],name=k,legendgroup=k, showlegend=False),1,2)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.revenue_yoy,name=k,line=line_,legendgroup=k, showlegend=False),2,1)
    fig.append_trace(go.Scatter(x=data_df.index, y=data_df.profit_dedt_yoy,name=k,line=line_,legendgroup=k, showlegend=False),2,2)

fig.update_layout(barmode='stack')
fig.show()

## 总资产、净资产增长率

In [24]:
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['total_ass_yoy'] = data_df.total_assets/data_df.total_assets.shift(1)
    data_df['total_hldr_eqy_inc_min_int_yoy'] = data_df.total_hldr_eqy_inc_min_int/data_df.total_hldr_eqy_inc_min_int.shift(1)


In [ ]:
## 这个对比的意义是？

# 利润质量

## 营业收入 vs 销售收现

1. total_revenue	营业总收入
2. c_fr_sale_sg	销售商品、提供劳务收到的现金
3. salescash_to_or	销售商品提供劳务收到的现金/营业收入

In [57]:
rows = int(np.ceil(len(data_dfs) / 2))
fig = make_subplots(rows,2, subplot_titles=list(data_dfs.keys()))

flag = True
for i,k in enumerate(data_dfs.keys()):
    r = i // 2 + 1
    c = i % 2 + 1
    data_df = data_dfs[k]
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_revenue,line=dict(color=DEFAULT_PLOTLY_COLORS[0]),name='营业总收入',showlegend=flag),r,c)
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.c_fr_sale_sg,line=dict(color=DEFAULT_PLOTLY_COLORS[1]),name='销售收现',showlegend=flag),r,c)
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.total_ass_yoy ,line=dict(color=DEFAULT_PLOTLY_COLORS[2]),name='销售收现/营业收入',showlegend=flag),r,c)
    flag = False

comp_length = len(data_dfs)
with fig.batch_update():
    for i in range(comp_length):
        fig.data[3*i+2]['yaxis'] = f'y{comp_length+i+1}'
        fig.layout[f'yaxis{comp_length+i+1}'] = dict(overlaying=f"y{'' if i==0 else i+1}", anchor=f"x{'' if i==0 else i+1}", side='right', showgrid=False)
fig.show()

## 净利润 vs 经营现金流净额

1. profit_dedt	扣除非经常性损益后的净利润
2. n_income	净利润(含少数股东损益)
3. n_cashflow_act	经营活动产生的现金流量净额


In [58]:
rows = int(np.ceil(len(data_dfs) / 2))
fig = make_subplots(rows,2, subplot_titles=list(data_dfs.keys()))

flag = True
for i,k in enumerate(data_dfs.keys()):
    r = i // 2 + 1
    c = i % 2 + 1
    data_df = data_dfs[k]
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.profit_dedt,line=dict(color=DEFAULT_PLOTLY_COLORS[0]),name='扣非净利',showlegend=flag),r,c)
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.n_cashflow_act,line=dict(color=DEFAULT_PLOTLY_COLORS[1]),name='经营现金净额',showlegend=flag),r,c)
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df.profit_dedt_to_cha ,line=dict(color=DEFAULT_PLOTLY_COLORS[2]),name='经营现金净额/扣非净利',showlegend=flag),r,c)
    fig.update_layout(yaxis5=dict(anchor='x', overlaying='y', side='right',domain=[0,1]))#设置坐标轴的格式，一般次坐标轴在右侧
    flag = False
    
comp_length = len(data_dfs)
with fig.batch_update():
    for i in range(comp_length):
        fig.data[3*i+2]['yaxis'] = f'y{comp_length+i+1}'
        fig.layout[f'yaxis{comp_length+i+1}'] = dict(overlaying=f"y{'' if i==0 else i+1}", anchor=f"x{'' if i==0 else i+1}", side='right', showgrid=False)
fig.show()

# 管理层能力

## 应收账款周转率

In [60]:
fig = go.Figure(layout=dict(title='应收账款周转率'))
for k in data_dfs:
    data_df = data_dfs[k]
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['ar_turn'],name=k))
fig.show()


## 资产周转率

In [61]:
cols_turn = {"inv_turn":"存货周转率",
"ca_turn":"流动资产周转率",
"fa_turn":"固定资产周转率",
"assets_turn":"总资产周转率"}

In [62]:
rows = int(np.ceil(len(data_dfs) / 2))
fig = make_subplots(rows,2, subplot_titles=list(data_dfs.keys()))

flag = True
for i,k in enumerate(data_dfs.keys()):
    r = i // 2 + 1
    c = i % 2 + 1
    data_df = data_dfs[k]
    data_df['inv_turn'] = data_df['revenue'] / data_df['inventories']
    
    for j,col in enumerate(cols_turn):
        fig.add_trace(go.Scatter(x=data_df.index, y=data_df[col],line=dict(color=DEFAULT_PLOTLY_COLORS[j]),name=cols_turn[col],showlegend=flag),r,c)
        
    flag = False

fig.show()

# 资产负债表

## 货币资金的比率

In [63]:
fig = go.Figure(layout=dict(title='货币资金比率'))
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['money_cap_ratio'] = data_df['money_cap']/data_df['total_assets']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['money_cap_ratio'],name=k))
fig.show()

## 经营资产比率

In [64]:
cols_op = {"notes_receiv":"应收票据",
"accounts_receiv":"应收账款",
"oth_receiv":"其他应收款",
"prepayment":"预付款项",
"div_receiv":"应收股利",
"int_receiv":"应收利息",
"inventories":"存货",
"produc_bio_assets":"生产性生物资产"}

In [65]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['op_ratio'] = data_df[cols_op.keys()].sum(axis=1)/data_df['total_assets']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['op_ratio'],name=k))
fig.show()


### 经营资产 vs 营收

In [66]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['op_ratio_to_revenue'] = data_df[cols_op.keys()].sum(axis=1)/data_df['total_revenue']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['op_ratio_to_revenue'],name=k))
fig.show()
# 越小越好

## 生产资产

In [67]:
cols_prod = {"fix_assets":"固定资产",
"cip":"在建工程",
"const_materials":"工程物资",
"intan_assets":"无形资产",
"goodwill":"商誉",
"lt_amor_exp":"长期待摊费用",
"defer_tax_assets":"递延所得税资产"}
# ？递延所得税资产 or 负债？

In [68]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['prod_ratio'] = data_df[cols_prod.keys()].sum(axis=1)/data_df['total_assets']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['prod_ratio'],name=k))
fig.show()


### 生产资产 vs 营收

In [69]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['prod_ratio_to_reve'] = data_df[cols_prod.keys()].sum(axis=1)/data_df['total_revenue']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['prod_ratio_to_reve'],name=k))
fig.show()


## 轻重比

In [70]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['opp_to_prodass'] = data_df['operate_profit']/data_df[cols_prod.keys()].sum(axis=1)
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['opp_to_prodass'],name=k))
fig.show()

## 非主营相关资产

In [71]:
cols_oth_ass = {
"trad_asset":"交易性金融资产",
"pur_resale_fa":"买入返售金融资产",
"oth_cur_assets":"其他流动资产",
"fa_avail_for_sale":"可供出售金融资产",
"htm_invest":"持有至到期投资",
"lt_eqt_invest":"长期股权投资",
"invest_real_estate":"投资性房地产"}

In [72]:
fig = go.Figure()
for k in data_dfs:
    data_df = data_dfs[k]
    data_df['oth_ass_ratio'] = data_df[cols_oth_ass.keys()].sum(axis=1)/data_df['total_assets']
    fig.add_trace(go.Scatter(x=data_df.index, y=data_df['oth_ass_ratio'],name=k))
fig.show()


# 现金流量表

## 三种现金流量覆盖情况

In [73]:
rows = int(np.ceil(len(data_dfs) / 2))
fig = make_subplots(rows,2, subplot_titles=list(data_dfs.keys()))

flag = True
for i,k in enumerate(data_dfs.keys()):
    r = i // 2 + 1
    c = i % 2 + 1
    data_df = data_dfs[k]
    fig.add_trace(go.Bar(x=data_df.index, y=data_df.n_cashflow_act,marker_color=DEFAULT_PLOTLY_COLORS[0],offsetgroup=0, name='经营活动现金流净额',showlegend=flag),r,c)
    fig.add_trace(go.Bar(x=data_df.index, y=-data_df.n_cashflow_inv_act,marker_color=DEFAULT_PLOTLY_COLORS[1],offsetgroup=1, name='投资活动现金流净额',showlegend=flag),r,c)
    fig.add_trace(go.Bar(x=data_df.index, y=-data_df.n_cash_flows_fnc_act ,marker_color=DEFAULT_PLOTLY_COLORS[2],offsetgroup=1, name='融资活动现金流净额',showlegend=flag),r,c)

    flag = False
fig.show()

## 经营活动现金流向上为正
## 投资与融资向上代表负的，为的是体现出经营现金流是否可以覆盖投资、融资现金流支出
## 向下代表正，投资表示收到前，融资表示在借钱


In [43]:
data_df[['n_cashflow_act','n_cashflow_inv_act','n_cash_flows_fnc_act']]

,n_cashflow_act,n_cashflow_inv_act,n_cash_flows_fnc_act
date,,,
2020-12-31,397879.0,447283.0,-550491.0
2019-12-31,679789.0,-129322.0,-482315.0
2018-12-31,905675.0,-334934.0,-384136.0
2017-12-31,688317.0,-416102.0,-341712.0
2016-12-31,740504.0,-596433.0,-369708.0
2015-12-31,583636.0,-367630.0,-179792.0
2014-12-31,271180.0,-344706.0,-277115.0
2013-12-31,318001.0,-166530.0,-131867.0
2012-12-31,549970.0,-358793.0,-135081.0


# 字典转化

In [ ]:
text = """
000672 上峰水泥
600585 海螺水泥
300552 万集科技
002869 金溢科技
601318 中国平安
"""
for line in text.split("\n"):
    if "\t" not in line:continue
    k,c = line.split("\t")
    print(f'"{k}":"{c}",')

In [ ]:
text = """
000672 上峰水泥
600585 海螺水泥
300552 万集科技
002869 金溢科技
601318 中国平安
"""
for line in text.split("\n"):
    if len(line) < 3: continue
    k,c = line.split()
    print(f'"{k}":"{c}",')